In [7]:
import tensorflow as tf
import numpy as np 
from tensorflow.examples.tutorials.mnist import input_data
FLAGS= None

In [8]:
mnist = input_data.read_data_sets("./MNIST_Data", one_hot=False)

Extracting ./MNIST_Data\train-images-idx3-ubyte.gz
Extracting ./MNIST_Data\train-labels-idx1-ubyte.gz
Extracting ./MNIST_Data\t10k-images-idx3-ubyte.gz
Extracting ./MNIST_Data\t10k-labels-idx1-ubyte.gz


In [5]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

flatx=tf.placeholder(tf.float32, [None, 784])
x=tf.reshape(flatx,[-1, 28, 28, 1])#what is the last input and why just x as the first 

with tf.variable_scope('FirstConv'): 
    
    weight=tf.get_variable("W", [10, 10, 1, 32], 
                          initializer=tf.truncated_normal_initializer(stddev=np.sqrt(2/784)))
    bias=tf.get_variable("b",initializer=tf.constant(.01,shape=[32]))
    
    z=tf.nn.conv2d(x, weight, strides=[1,1,1,1], padding="SAME") + bias
    #computes a 2d convolution on 4D input, what does that mean
    
    a1=tf.nn.relu(z)
    
    
with tf.variable_scope('pool1'):
    pooled1=tf.nn.max_pool(a1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
    
with tf.variable_scope('SecondConv'): 
    weight=tf.get_variable("W", [5, 5, 32, 64], 
                          initializer=tf.truncated_normal_initializer(stddev=np.sqrt(2/64)))
    #how do we decide initializers and what is with the shapes
    
    bias= tf.get_variable("b",initializer=tf.constant(.01,shape=[64]))
    #why has shape gone up when pooling shrinks the data size
    
    z=tf.nn.conv2d(pooled1, weight, strides=[1,1,1,1], padding="SAME") + bias
    
    a2=tf.nn.relu(z)
    
    
with tf.variable_scope('pool2'):
    pooled2=tf.nn.max_pool(a2, ksize=[1,2,2,1], strides=[1,1,1,1], padding="SAME")
    
with tf.variable_scope('FullyConnectedLayer1'): 
    flat_size=int(np.prod(pooled2.shape[1:]))
    flatpool=tf.reshape(pooled2, [-1, flat_size])
    
    weight=tf.get_variable("W", [flat_size, 1024], 
                          initializer=tf.truncated_normal_initializer(stddev=np.sqrt(2/flat_size)))
    #why is it size 1024 the original flattened image wasnt even that large
    
    bias=tf.get_variable("b",initializer=tf.constant(.01,shape=[1024]))
    
    z=tf.matmul(flatpool, weight)+bias
    
    a3=tf.nn.relu(z)
    
    p1 = tf.Print(tf.shape(a3), [tf.shape(a3)])
    
with tf.variable_scope('FullyConnectedLayer2'):
    weight=tf.get_variable("W", [1024, 10], 
                          initializer=tf.truncated_normal_initializer(stddev=np.sqrt(2/1024)))
    bias= tf.get_variable("b",initializer=tf.constant(.01,shape=[10]))
    
    z=tf.matmul(a3, weight)+bias
    
    hyp=tf.nn.softmax(z)
    
    
y=tf.placeholder(tf.int64, [None])

loss= tf.losses.sparse_softmax_cross_entropy(labels=y, logits=hyp)
#what is this loss function
training_step= tf.train.GradientDescentOptimizer(.005).minimize(loss)
tf.global_variables_initializer().run()


In [9]:
for i in range(5000): 
    batch_x, batch_y= mnist.train.next_batch(100)
            
    sess.run(training_step, feed_dict={flatx:batch_x, y: batch_y})
    #unclear about batches
    
    if i%100 == 0:
        correct_prediction = tf.equal(tf.argmax(hyp, 1), y)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print(sess.run(
          accuracy, feed_dict={
              flatx: mnist.test.images[:100],
              y: mnist.test.labels[:100]
          }))

0.91
0.91
0.91
0.92
0.91
0.92
0.92
0.91
0.91
0.91
0.91
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
0.92
